In [ ]:
# Plot the most common pairs

pairs_list_flat = [(i[0]+', '+i[1]) for i in pairs_list_aligned]
pairs = pd.DataFrame(pairs_list_flat, columns=['pairs'])
plt.figure(figsize = (10,8))
ax = pairs['pairs'].value_counts().nlargest(20).plot.barh()
plt.show()

In [ ]:
# Plot the least common pairs

plt.figure(figsize = (10,8))
ax = pairs['pairs'].value_counts().nsmallest(20).plot.barh()
plt.show()